In [1]:
import os
import pickle
import numpy as np
import pandas as pd

In [2]:
working_dir = "/om2/user/ckauf/.result_caching/neural_nlp.models.wrapper.core.ActivationsExtractorHelper._from_sentences_stored"

## Example of what the structure looks like

In [4]:
file = os.path.join(working_dir, "identifier=glove,stimuli_identifier=Pereira2018-Original-384sentences7.pkl")
with open(file, 'rb') as f:
    result = pickle.load(f)

data = result['data']
#data

layers = np.unique(data.layer)
print(layers)
last_layer = layers[-1]
print(last_layer)
data[{"neuroid": [layer == last_layer for layer in data["layer"].values]}]
print(data)

['projection']
projection
<xarray.NeuroidAssembly (presentation: 4, neuroid: 300)>
array([[ 0.004457,  0.077929, -0.103699, ..., -0.261306, -0.007627,  0.194675],
       [-0.101523,  0.061935, -0.132709, ..., -0.363577,  0.166671,  0.197726],
       [-0.188426,  0.044563,  0.027355, ..., -0.218456,  0.014926,  0.176471],
       [-0.244006,  0.081785,  0.033568, ..., -0.583785,  0.257052,  0.147002]],
      dtype=float32)
Coordinates:
  * presentation       (presentation) MultiIndex
  - stimulus_sentence  (presentation) object 'a banana is a long fruit that grows in bunchs with a soft edible inside' ... 'unripe bananas and plantains are staple foods and often cooked like potatoes'
  - sentence_num       (presentation) int64 0 1 2 3
  * neuroid            (neuroid) MultiIndex
  - neuroid_num        (neuroid) int64 0 1 2 3 4 5 6 7 ... 23 24 25 26 27 28 29
  - model              (neuroid) object 'glove' 'glove' ... 'glove' 'glove'
  - layer              (neuroid) object 'projection' ... 'p

In [29]:
dfile = os.path.join(working_dir, "identifier=distilgpt2,stimuli_identifier=Pereira2018-Original-384sentences7.pkl")
with open(dfile, 'rb') as f:
    dresult = pickle.load(f)

ddata = dresult['data']
ddata.values.shape

(4, 5376)

In [5]:
data.shape
# number of sentences in passage * hidden layer activation

(4, 300)

In [6]:
np.unique(data.layer) 

array(['projection'], dtype=object)

## Create dictionary
### Structure:

For all models one separate dictionary structured as follows:
* key = sentenceIdentifier _ conditionIdentifier
* value = associated unpickled file (assembly like above)

In [7]:
glove_dic = {}
lm1b_dic = {}
distilgpt2_dic = {}

for ind,filename in enumerate(os.listdir(working_dir)):
    if ind%500 == 0:
        print(ind)
        
    if "identifier=glove" in filename:
        passage = filename.split(",")[-1].split("-")[-1].split(".")[0]
        condition = filename.split(",")[-1].split("-")[-2]
        #key = passage + "_" + condition
        file = os.path.join(working_dir,filename)
        with open(file, 'rb') as f:
            result = pickle.load(f)
        #glove_dic[key] = result
        if not condition in glove_dic:
            glove_dic[condition] = {}
        glove_dic[condition][passage] = result
        
    if "identifier=lm_1b" in filename:
        passage = filename.split(",")[-1].split("-")[-1].split(".")[0]
        condition = filename.split(",")[-1].split("-")[-2]
        #key = passage + "_" + condition
        file = os.path.join(working_dir,filename)
        with open(file, 'rb') as f:
            result = pickle.load(f)
        #lm1b_dic[key] = result
        if not condition in lm1b_dic:
            lm1b_dic[condition] = {}
        lm1b_dic[condition][passage] = result
        
    if "identifier=distilgpt2" in filename:
        passage = filename.split(",")[-1].split("-")[-1].split(".")[0]
        condition = filename.split(",")[-1].split("-")[-2]
        #key = passage + "_" + condition
        #print(key)
        file = os.path.join(working_dir,filename)
        with open(file, 'rb') as f:
            result = pickle.load(f)
        #distilgpt2_dic[key] = result
        if not condition in distilgpt2_dic:
            distilgpt2_dic[condition] = {}
        distilgpt2_dic[condition][passage] = result
print("Done")

0
500
1000
1500
2000
2500
3000
3500
4000
Done


In [8]:
print(glove_dic.keys())
print(len(glove_dic['Original']))

dict_keys(['Original', 'Scr1', 'Scr3', 'Scr5', 'Scr7', 'backward', 'lowPMI', 'random'])
168


In [9]:
print(len(glove_dic)) # old, from different iteration // keeping for sanity check

8


## Quick number sanity-check
* 8 conditions á 168 passage encodings >> 8 * 168 = 1344
* 
* 168 * 4 = 672 (Do all passages have 4 sentences?)
* 243 + 384 = 627 (No, some must have fewer!)

In [10]:
cond_dict = {}

for ind,filename in enumerate(os.listdir(working_dir)):
    if "identifier=glove" in filename:
        condition = filename.split(",")[-1].split("-")[-2]
        if not condition in cond_dict:
            cond_dict[condition] = 1
        else:
            cond_dict[condition] += 1
print(cond_dict)

{'Original': 168, 'Scr1': 168, 'Scr3': 168, 'Scr5': 168, 'Scr7': 168, 'backward': 168, 'lowPMI': 168, 'random': 168}


__________________________________________________________________________________________________________________

__________________________________________________________________________________________________________________


# Get correlations

### Questions
* size is passage_len * hidden_layer_size >> Should correlation value come from a matrix norm?
* alternatively, we could do independent plots for each layer.

__________________________________________________________________________________________________________________

### Quick change: Better dictionary structure: sentence to condition

In [3]:
glove_dic_senttocond = {}
lm1b_dic_senttocond = {}
distilgpt2_dic_senttocond = {}

for ind,filename in enumerate(os.listdir(working_dir)):
    if ind%500 == 0:
        print(ind)
        
    if "identifier=glove" in filename:
        passage = filename.split(",")[-1].split("-")[-1].split(".")[0]
        number = passage.split("sentences")[-1]
        if len(number) == 1:
            new_name = passage[:-1] + number.zfill(2)
        else:
            new_name = passage
        #print(new_name)
        condition = filename.split(",")[-1].split("-")[-2]
        #key = passage + "_" + condition
        file = os.path.join(working_dir,filename)
        with open(file, 'rb') as f:
            result = pickle.load(f)
        result = result['data']
        layers = np.unique(result.layer)
        final_layer = layers[-1]
        data = result[{"neuroid": [layer == final_layer for layer in result["layer"].values]}]
        
        if not new_name in glove_dic_senttocond:
            glove_dic_senttocond[new_name] = {}
        glove_dic_senttocond[new_name][condition] = data
        
    if "identifier=lm_1b" in filename:
        passage = filename.split(",")[-1].split("-")[-1].split(".")[0]
        number = passage.split("sentences")[-1]
        if len(number) == 1:
            new_name = passage[:-1] + number.zfill(2)
        else:
            new_name = passage
        condition = filename.split(",")[-1].split("-")[-2]
        #key = passage + "_" + condition
        file = os.path.join(working_dir,filename)
        with open(file, 'rb') as f:
            result = pickle.load(f)
        result = result['data']
        layers = np.unique(result.layer)
        final_layer = layers[-1]
        data = result[{"neuroid": [layer == final_layer for layer in result["layer"].values]}]

        if not new_name in lm1b_dic_senttocond:
            lm1b_dic_senttocond[new_name] = {}
        lm1b_dic_senttocond[new_name][condition] = data
        
    if "identifier=distilgpt2" in filename:
        passage = filename.split(",")[-1].split("-")[-1].split(".")[0]
        number = passage.split("sentences")[-1]
        if len(number) == 1:
            new_name = passage[:-1] + number.zfill(2)
        else:
            new_name = passage
        condition = filename.split(",")[-1].split("-")[-2]

        file = os.path.join(working_dir,filename)
        with open(file, 'rb') as f:
            result = pickle.load(f)
        result = result['data']
        layers = np.unique(result.layer)
        final_layer = layers[-1]
        data = result[{"neuroid": [layer == final_layer for layer in result["layer"].values]}]
        if not new_name in distilgpt2_dic_senttocond:
            distilgpt2_dic_senttocond[new_name] = {}
        distilgpt2_dic_senttocond[new_name][condition] = data
print("Done")

0
500
1000
1500
2000
2500
3000
3500
4000
Done


In [12]:
#result
#data = result[{"neuroid": [layer == "encoder.h.5" for layer in result["layer"].values]}]
#print(data)

In [13]:
# old code
#cols = df.corr(method='pearson').nlargest(k,target_col)[target_col].index
#cm = df[cols].corr(method='pearson')
#plt.figure(figsize=(15,8))
#plt.tight_layout()
#sns.heatmap(cm, annot=True, cmap = 'viridis')

In [4]:
distilgpt2_dic_senttocond['384sentences02']['Original'].values.shape

(4, 768)

# Get correlation heatmap

In [13]:
scr1 = []
scr3 = []
scr5 = []
scr7 = []
original = []
lowpmi = []
backward = []
random = []

sorted_dict = dict(sorted(distilgpt2_dic_senttocond.items()))
for key, value in sorted_dict.items(): #key is passage, value is dic from cond to xarray data
    original.append(value['Original'].values)
    scr1.append(value['Scr1'].values)
    scr3.append(value['Scr3'].values)
print(original)

[array([[ 0.07490777,  0.4333722 , -1.3863953 , ...,  0.2385128 ,
        -0.07436069,  0.11174872],
       [ 0.9501099 ,  0.10308348, -1.9391278 , ...,  0.3508047 ,
        -0.7217415 ,  0.02178628],
       [-0.1226256 ,  0.1458139 , -2.1622283 , ...,  0.53761   ,
         0.14394495,  0.17006397],
       [-0.31725436,  0.5727599 , -1.1838961 , ...,  0.34671035,
        -0.36449128, -0.25676194]], dtype=float32), array([[-0.09892738, -0.40661898, -1.0095222 , ...,  0.03841835,
        -0.24056432,  0.26098356],
       [-0.19813704,  0.41001764, -0.4742375 , ...,  0.26721087,
         0.08068482,  0.2485303 ],
       [ 0.5078993 ,  0.51993537, -0.25285086, ...,  0.02374682,
        -0.02693333,  0.00361259],
       [ 0.44988915,  0.22143011, -2.1330822 , ...,  0.28167564,
        -0.4411549 , -0.35658962]], dtype=float32), array([[-0.6102434 , -0.20404448, -1.8177676 , ...,  0.2716304 ,
        -0.07121444,  0.16647626],
       [ 0.36560136,  0.2499693 , -1.0417398 , ...,  0.43521518,


In [6]:
print(len(original))
print(type(original[0]))
print(len(original[0]))
print(len(original[0][0]))

168
<class 'numpy.ndarray'>
4
768


In [14]:
original_flatten = [item for sublist in original for item in sublist]
scr1_flatten = [item for sublist in scr1 for item in sublist]
scr3_flatten = [item for sublist in scr3 for item in sublist]

In [12]:
print(len(scr1_flatten))

627


In [34]:
#df = pd.DataFrame(data={'Original':original_flatten, 'Scr1':scr1_flatten, 'Scr3':scr3_flatten})
df1 = pd.DataFrame(data={'Original':original_flatten})
df2 = pd.DataFrame(data={'Scr1':scr1_flatten})
correlations = np.corrcoef(original_flatten, scr1_flatten)
print(np.shape(correlations))

(1254, 1254)


In [27]:
df1

,Original
0,"[0.07490777, 0.4333722, -1.3863953, -0.0211150..."
1,"[0.9501099, 0.103083484, -1.9391278, 0.0482755..."
2,"[-0.1226256, 0.1458139, -2.1622283, 0.48651263..."
3,"[-0.31725436, 0.5727599, -1.1838961, 0.0313818..."
4,"[-0.09892738, -0.40661898, -1.0095222, -0.0966..."
...,...
622,"[0.21522328, -0.077501625, -1.172828, 0.657538..."
623,"[0.15427107, 0.879871, -0.7449374, -0.08911968..."
624,"[-0.09369865, -0.0902057, -0.93472385, -0.0739..."
625,"[-0.028594162, 0.60758823, -0.56496173, 0.2645..."
